In [10]:
import pandas as pd
import numpy as np
import sys, os
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,LancasterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from textblob import TextBlob
from google.colab import drive
from pathlib import Path
import logging
import string
nltk.download('reuters') # Downloading corpus
nltk.download('stopwords') # Downloading stopwords
nltk.download('punkt') # Downloading tokenizer
nltk.download('wordnet')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
from google.colab import files
uploaded = files.upload()

Saving input_data.xlsx to input_data.xlsx


In [12]:
logging.basicConfig(filename='app.log', filemode='w', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
pd.set_option('display.max_colwidth', 0)
#print(Path('app.log').read_text())

## Read ticket data file as data frame

In [107]:
ticket_df = pd.read_excel('input_data.xlsx')

# Show head of the data

In [108]:
ticket_df.head()

,Short description,Description,Caller,Assignment group
0,login issue,-verified user details.(employee# & manager name)\r\n-checked the user name in ad and reset the password.\r\n-advised the user to login and check.\r\n-caller confirmed that he was able to login.\r\n-issue resolved.,spxjnwir pjlcoqds,GRP_0
1,outlook,"\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail.com\r\n\r\nhello team,\r\n\r\nmy meetings/skype meetings etc are not appearing in my outlook calendar, can somebody please advise how to correct this?\r\n\r\nkind",hmjdrvpb komuaywn,GRP_0
2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail.com\r\n\r\nhi\r\n\r\ni cannot log on to vpn\r\n\r\nbest,eylqgodm ybqkwiam,GRP_0
3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0


# Shape of the data

In [109]:
ticket_df.shape

(8500, 4)

## Describe data

In [110]:
ticket_df.describe()

,Short description,Description,Caller,Assignment group
count,8492,8499,8500,8500
unique,7481,7817,2950,74
top,password reset,the,bpctwhsn kzqsbmtp,GRP_0
freq,38,56,810,3976


## Find any Null values in the data 

In [111]:
ticket_df.isnull().sum()

Short description    8
Description          1
Caller               0
Assignment group     0
dtype: int64

In [115]:
ticket_df = ticket_df.fillna("")

In [116]:
ticket_df.isnull().sum()

Short description    0
Description          0
Caller               0
Assignment group     0
dtype: int64

In [117]:
targetClassCnt=ticket_df['Assignment group'].value_counts()
targetClassCnt.describe()

count    74.000000  
mean     114.864865 
std      465.747516 
min      1.000000   
25%      5.250000   
50%      26.000000  
75%      84.000000  
max      3976.000000
Name: Assignment group, dtype: float64

In [118]:
duplicate = ticket_df[ticket_df.duplicated()]
duplicate.head()

,Short description,Description,Caller,Assignment group
51,call for ecwtrjnq jpecxuty,call for ecwtrjnq jpecxuty,olckhmvx pcqobjnd,GRP_0
229,call for ecwtrjnq jpecxuty,call for ecwtrjnq jpecxuty,olckhmvx pcqobjnd,GRP_0
493,ticket update on inplant_872730,ticket update on inplant_872730,fumkcsji sarmtlhy,GRP_0
512,blank call //gso,blank call //gso,rbozivdq gmlhrtvp,GRP_0
667,job bkbackup_tool_powder_prod_full failed in job_scheduler at: 10/22/2016 18:33:00,received from: monitoring_tool@company.com\r\n\r\njob bkbackup_tool_powder_prod_full failed in job_scheduler at: 10/22/2016 18:33:00,bpctwhsn kzqsbmtp,GRP_8


In [119]:
ticket_df.drop_duplicates(inplace=True)

In [120]:
ticket_df.shape

(8417, 4)

In [129]:
class PreProcessing:
    lemmatizer=""
    stemmer =""
    tokenizer=""
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stemmer = LancasterStemmer()
        self.tokenizer = RegexpTokenizer(r'\w+')
    
    def data_clean(self, sentence):
      if pd.isnull(sentence):
        return sentence
      sentence= sentence.lower()
      sentence = ' '.join([w for w in sentence.split() if not self.is_valid_date(w)])
      sentence = re.sub(r"received from:",' ',sentence)
      sentence = re.sub(r"select the following link to view the disclaimer in an alternate language.",' ',sentence)
      sentence = re.sub(r"from:",' ',sentence)
      sentence = re.sub(r"to:",' ',sentence)
      sentence = re.sub(r"subject:",' ',sentence)
      sentence = re.sub(r"re:",' ',sentence)
      sentence = re.sub(r"fw:",' ',sentence)
      sentence = re.sub(r"sent:",' ',sentence)
      sentence = re.sub(r"ic:",' ',sentence)
      sentence = re.sub(r"cc:",' ',sentence)
      sentence = re.sub(r"bcc:",' ',sentence)
      #Remove email 
      sentence = re.sub(r'\S*@\S*\s?', '', sentence)
      # Remove underscores
      sentence = re.sub(r'\_',' ', sentence)

      # Remove new line characters 
      sentence = re.sub(r'\n',' ',sentence)
      # Remove hashtag while keeping hashtag sentence
      sentence = re.sub(r'#','', sentence)
      #& 
      sentence = re.sub(r'&;?', 'and',sentence)
      # Remove HTML special entities (e.g. &amp;)
      sentence = re.sub(r'\&\w*;', '', sentence)
      # Remove hyperlinks
      sentence = re.sub(r'https?:\/\/.*\/\w*', '', sentence)  
      # Remove characters beyond Readable formart by Unicode:
      sentence= ''.join(c for c in sentence if c <= '\uFFFF') 
      sentence = sentence.strip()
      # Remove unreadable characters  (also extra spaces)
      sentence = ' '.join(re.sub("[^\u0030-\u0039\u0041-\u005a\u0061-\u007a]", " ", sentence).split())

      # Single character      
      sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
      sentence = sentence.strip()
      return sentence
    
    def is_valid_date(self,date_str):
      try:
        parser.parse(date_str)
        return True
      except:
        return False
    
    def mergeColumns(self, df):
        df['desc']=df['Short description'] + " " + df['Description'] + " " + df['Caller']
        df.drop("Short description", axis=1, inplace=True)
        df.drop("Description", axis=1, inplace=True)
        df.drop("Caller", axis=1, inplace=True)
        return df

    def remove_punctuation(self, sentence):
      try:
        no_punct = "".join([c for c in sentence if c not in string.punctuation])
        return no_punct
      except:
        print("remove_punctuation: sentence,",sentence)
        return sentence

    def make_tokens(self, sentence):
      return self.tokenizer.tokenize(sentence.lower())

    def remove_stopwords(self, words):
      return [w for w in words if w not in stopwords.words('english')]

    def word_lemmatize(self, words):
      return [self.lemmatizer.lemmatize(item) for item in words]


preProcessing=PreProcessing()

In [122]:
ticket_df=preProcessing.mergeColumns(ticket_df)
ticket_df['desc_clean'] = ticket_df['desc'].apply(preProcessing.data_clean)

In [123]:
ticket_df.head()

,Assignment group,desc,desc_clean
0,GRP_0,login issue -verified user details.(employee# & manager name)\r\n-checked the user name in ad and reset the password.\r\n-advised the user to login and check.\r\n-caller confirmed that he was able to login.\r\n-issue resolved. spxjnwir pjlcoqds,login issue verified user details employee and manager name checked the user name in ad and reset the password advised the user to login and check caller confirmed that he was able to login issue resolved spxjnwir pjlcoqds
1,GRP_0,"outlook \r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail.com\r\n\r\nhello team,\r\n\r\nmy meetings/skype meetings etc are not appearing in my outlook calendar, can somebody please advise how to correct this?\r\n\r\nkind hmjdrvpb komuaywn",outlook hello team my meetings skype meetings etc are not appearing in my outlook calendar can somebody please advise how to correct this kind hmjdrvpb komuaywn
2,GRP_0,cant log in to vpn \r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail.com\r\n\r\nhi\r\n\r\ni cannot log on to vpn\r\n\r\nbest eylqgodm ybqkwiam,cant log in to vpn hi cannot log on to vpn best eylqgodm ybqkwiam
3,GRP_0,unable to access hr_tool page unable to access hr_tool page xbkucsvz gcpydteq,unable to access hr tool page unable to access hr tool page xbkucsvz gcpydteq
4,GRP_0,skype error skype error owlgqjme qhcozdfx,skype error skype error owlgqjme qhcozdfx


In [124]:
#logging.getLogger().setLevel(logging.DEBUG)
#cleaning.preprocess(ticket_df['Description'][1])
#logging.getLogger().setLevel(logging.INFO)
# ticket_df['polarity'] = ticket_df['desc_clean'].map(lambda text: TextBlob(text).sentiment.polarity)
ticket_df['review_len'] = ticket_df['desc_clean'].astype(str).apply(len)
ticket_df['word_count'] = ticket_df['desc_clean'].apply(lambda x: len(str(x).split()))

In [125]:
ticket_df.head(50)

,Assignment group,desc,desc_clean,review_len,word_count
0,GRP_0,login issue -verified user details.(employee# & manager name)\r\n-checked the user name in ad and reset the password.\r\n-advised the user to login and check.\r\n-caller confirmed that he was able to login.\r\n-issue resolved. spxjnwir pjlcoqds,login issue verified user details employee and manager name checked the user name in ad and reset the password advised the user to login and check caller confirmed that he was able to login issue resolved spxjnwir pjlcoqds,222,38
1,GRP_0,"outlook \r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail.com\r\n\r\nhello team,\r\n\r\nmy meetings/skype meetings etc are not appearing in my outlook calendar, can somebody please advise how to correct this?\r\n\r\nkind hmjdrvpb komuaywn",outlook hello team my meetings skype meetings etc are not appearing in my outlook calendar can somebody please advise how to correct this kind hmjdrvpb komuaywn,160,26
2,GRP_0,cant log in to vpn \r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail.com\r\n\r\nhi\r\n\r\ni cannot log on to vpn\r\n\r\nbest eylqgodm ybqkwiam,cant log in to vpn hi cannot log on to vpn best eylqgodm ybqkwiam,65,14
3,GRP_0,unable to access hr_tool page unable to access hr_tool page xbkucsvz gcpydteq,unable to access hr tool page unable to access hr tool page xbkucsvz gcpydteq,77,14
4,GRP_0,skype error skype error owlgqjme qhcozdfx,skype error skype error owlgqjme qhcozdfx,41,6
5,GRP_0,unable to log in to engineering tool and skype unable to log in to engineering tool and skype eflahbxn ltdgrvkz,unable to log in to engineering tool and skype unable to log in to engineering tool and skype eflahbxn ltdgrvkz,111,20
6,GRP_1,"event: critical:HostName_221.company.com the value of mountpoint threshold for /oracle/SID_37/erpdata21/ event: critical:HostName_221.company.com the value of mountpoint threshold for /oracle/SID_37/erpdata21/sr3psa1d_7/sr3psa1d.data7,perpsr3psa1d,4524 is 98 jyoqwxhz clhxsoqy",event critical hostname 221 company com the value of mountpoint threshold for oracle sid 37 erpdata21 event critical hostname 221 company com the value of mountpoint threshold for oracle sid 37 erpdata21 sr3psa1d 7 sr3psa1d data7 perpsr3psa1d 4524 is 98 jyoqwxhz clhxsoqy,271,42
7,GRP_0,ticket_no1550391- employment status - new non-employee [enter user's name] ticket_no1550391- employment status - new non-employee [enter user's name] eqzibjhw ymebpoih,ticket no1550391 employment status new non employee enter user name ticket no1550391 employment status new non employee enter user name eqzibjhw ymebpoih,153,22
8,GRP_0,unable to disable add ins on outlook unable to disable add ins on outlook mdbegvct dbvichlg,unable to disable add ins on outlook unable to disable add ins on outlook mdbegvct dbvichlg,91,16
9,GRP_0,ticket update on inplant_874773 ticket update on inplant_874773 fumkcsji sarmtlhy,ticket update on inplant 874773 ticket update on inplant 874773 fumkcsji sarmtlhy,81,12


In [130]:
ticket_df['desc_punct'] = ticket_df['desc_clean'].map(preProcessing.remove_punctuation)
ticket_df['desc_token'] = ticket_df['desc_punct'].map(preProcessing.make_tokens)
ticket_df['desc_stop'] = ticket_df['desc_token'].map(preProcessing.remove_stopwords)
ticket_df['desc_lemma'] = ticket_df['desc_stop'].map(preProcessing.word_lemmatize)

In [131]:
ticket_df.head()

,Assignment group,desc,desc_clean,review_len,word_count,desc_punct,desc_token,desc_stop,desc_lemma
0,GRP_0,login issue -verified user details.(employee# & manager name)\r\n-checked the user name in ad and reset the password.\r\n-advised the user to login and check.\r\n-caller confirmed that he was able to login.\r\n-issue resolved. spxjnwir pjlcoqds,login issue verified user details employee and manager name checked the user name in ad and reset the password advised the user to login and check caller confirmed that he was able to login issue resolved spxjnwir pjlcoqds,222,38,login issue verified user details employee and manager name checked the user name in ad and reset the password advised the user to login and check caller confirmed that he was able to login issue resolved spxjnwir pjlcoqds,"[login, issue, verified, user, details, employee, and, manager, name, checked, the, user, name, in, ad, and, reset, the, password, advised, the, user, to, login, and, check, caller, confirmed, that, he, was, able, to, login, issue, resolved, spxjnwir, pjlcoqds]","[login, issue, verified, user, details, employee, manager, name, checked, user, name, ad, reset, password, advised, user, login, check, caller, confirmed, able, login, issue, resolved, spxjnwir, pjlcoqds]","[login, issue, verified, user, detail, employee, manager, name, checked, user, name, ad, reset, password, advised, user, login, check, caller, confirmed, able, login, issue, resolved, spxjnwir, pjlcoqds]"
1,GRP_0,"outlook \r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail.com\r\n\r\nhello team,\r\n\r\nmy meetings/skype meetings etc are not appearing in my outlook calendar, can somebody please advise how to correct this?\r\n\r\nkind hmjdrvpb komuaywn",outlook hello team my meetings skype meetings etc are not appearing in my outlook calendar can somebody please advise how to correct this kind hmjdrvpb komuaywn,160,26,outlook hello team my meetings skype meetings etc are not appearing in my outlook calendar can somebody please advise how to correct this kind hmjdrvpb komuaywn,"[outlook, hello, team, my, meetings, skype, meetings, etc, are, not, appearing, in, my, outlook, calendar, can, somebody, please, advise, how, to, correct, this, kind, hmjdrvpb, komuaywn]","[outlook, hello, team, meetings, skype, meetings, etc, appearing, outlook, calendar, somebody, please, advise, correct, kind, hmjdrvpb, komuaywn]","[outlook, hello, team, meeting, skype, meeting, etc, appearing, outlook, calendar, somebody, please, advise, correct, kind, hmjdrvpb, komuaywn]"
2,GRP_0,cant log in to vpn \r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail.com\r\n\r\nhi\r\n\r\ni cannot log on to vpn\r\n\r\nbest eylqgodm ybqkwiam,cant log in to vpn hi cannot log on to vpn best eylqgodm ybqkwiam,65,14,cant log in to vpn hi cannot log on to vpn best eylqgodm ybqkwiam,"[cant, log, in, to, vpn, hi, cannot, log, on, to, vpn, best, eylqgodm, ybqkwiam]","[cant, log, vpn, hi, cannot, log, vpn, best, eylqgodm, ybqkwiam]","[cant, log, vpn, hi, cannot, log, vpn, best, eylqgodm, ybqkwiam]"
3,GRP_0,unable to access hr_tool page unable to access hr_tool page xbkucsvz gcpydteq,unable to access hr tool page unable to access hr tool page xbkucsvz gcpydteq,77,14,unable to access hr tool page unable to access hr tool page xbkucsvz gcpydteq,"[unable, to, access, hr, tool, page, unable, to, access, hr, tool, page, xbkucsvz, gcpydteq]","[unable, access, hr, tool, page, unable, access, hr, tool, page, xbkucsvz, gcpydteq]","[unable, access, hr, tool, page, unable, access, hr, tool, page, xbkucsvz, gcpydteq]"
4,GRP_0,skype error skype error owlgqjme qhcozdfx,skype error skype error owlgqjme qhcozdfx,41,6,skype error skype error owlgqjme qhcozdfx,"[skype, error, skype, error, owlgqjme, qhcozdfx]","[skype, error, skype, error, owlgqjme, qhcozdfx]","[skype, error, skype, error, owlgqjme, qhcozdfx]"
